In [1]:
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datetime import datetime
from google.colab import drive
import os, zipfile, glob
import pickle

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Spanish books from the gutenberg.org for text generation (top 5 in volume with the right encoding)
'2000-0'
'62870-0'
'64465-0'
'59539-0'
'57654-0'

In [7]:
!wget "https://www.gutenberg.org/files/2000/2000-0.txt"

--2022-01-17 23:21:40--  https://www.gutenberg.org/files/2000/2000-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2226045 (2.1M) [text/plain]
Saving to: ‘2000-0.txt.3’

2000-0.txt.3        100%[===================>]   2.12M  1.63MB/s    in 1.3s    

2022-01-17 23:21:42 (1.63 MB/s) - ‘2000-0.txt.3’ saved [2226045/2226045]



In [8]:
!wget "https://www.gutenberg.org/files/62870/62870-0.txt"

--2022-01-17 23:21:43--  https://www.gutenberg.org/files/62870/62870-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1870046 (1.8M) [text/plain]
Saving to: ‘62870-0.txt.3’

62870-0.txt.3       100%[===================>]   1.78M  1.37MB/s    in 1.3s    

2022-01-17 23:21:45 (1.37 MB/s) - ‘62870-0.txt.3’ saved [1870046/1870046]



In [9]:
!wget "https://www.gutenberg.org/files/64465/64465-0.txt"

--2022-01-17 23:21:45--  https://www.gutenberg.org/files/64465/64465-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1397772 (1.3M) [text/plain]
Saving to: ‘64465-0.txt.3’

64465-0.txt.3       100%[===================>]   1.33M  1.19MB/s    in 1.1s    

2022-01-17 23:21:47 (1.19 MB/s) - ‘64465-0.txt.3’ saved [1397772/1397772]



In [10]:
!wget "https://www.gutenberg.org/files/59539/59539-0.txt"

--2022-01-17 23:21:47--  https://www.gutenberg.org/files/59539/59539-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1252731 (1.2M) [text/plain]
Saving to: ‘59539-0.txt.3’

59539-0.txt.3       100%[===================>]   1.19M  1.07MB/s    in 1.1s    

2022-01-17 23:21:49 (1.07 MB/s) - ‘59539-0.txt.3’ saved [1252731/1252731]



In [11]:
!wget "https://www.gutenberg.org/files/57654/57654-0.txt"

--2022-01-17 23:21:49--  https://www.gutenberg.org/files/57654/57654-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1131399 (1.1M) [text/plain]
Saving to: ‘57654-0.txt.3’

57654-0.txt.3       100%[===================>]   1.08M   991KB/s    in 1.1s    

2022-01-17 23:21:52 (991 KB/s) - ‘57654-0.txt.3’ saved [1131399/1131399]



In [12]:
!pip install unidecode

In [13]:
import codecs
from bs4 import BeautifulSoup
import unidecode

In [14]:
list_books = ['2000-0', '62870-0', '64465-0', '59539-0', '57654-0']
text_add = ''
for b in list_books:
  with open(b + '.txt', 'r') as f:
    text = f.read()

    info_beg = text.find('Title: ')
    begin = text.find('*** START OF THIS PROJECT GUTENBERG EBOOK ')
    title = ''
    if begin > -1:
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 42 + len(title) + 3
                
    else:
      begin = text.find('START OF THE PROJECT GUTENBERG EBOOK ')
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 37 + len(title) + 3

    end = text.find('*** END OF THIS PROJECT GUTENBERG EBOOK')
    if end == -1:
      end = text.find('*** END')
    text = text[begin_2 : end]

    #remove html tags from text
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
    
    #remove accented characters from text, e.g. café
    text = unidecode.unidecode(text)

    text_add = text_add + text + '\n'

In [15]:
len(text.split(' '))

172417

In [16]:
# Create two dictionaries:
# int2char -- maps integers to characters
# char2int -- maps characters to unique integers

int2char = dict(enumerate(set(text)))
char2int = { val: idx  for idx, val in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [17]:
len(char2int)

82

In [18]:
def one_hot_encode(arr, n_labels):
    
    # arr - array of integers
    # n_labels - number of labels (the size of a one-hot-encoded vector)

    one_hot = np.eye(n_labels)[arr]
    
    return one_hot

In [19]:
# check that the function works correctly
test_indx = np.array([[7, 2, 5]])
one_hot = one_hot_encode(test_indx, 8)

print(one_hot)

[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]


In [20]:
def get_batches(arr, batch_size, seq_length):
    # Create a generator that returns batches of size batch_size x seq_length
    
    batch_size_total = batch_size * seq_length
    ## Get the number of batches we can make
    n_batches = int(len(arr) / batch_size_total)
    
    ## Keep only enough characters to make full batches
    arr = arr[: n_batches*batch_size_total] 
    
    ## Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1)) 
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n : n + seq_length ]
        # The target is a version of x shifted by one (do not forget border conditions)
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        if n + seq_length < arr.shape[1]:
            y[:, -1] = arr[:, n + seq_length]
        else:
            arr[:, 0]
        yield x, y

In [21]:
batches = get_batches(encoded, 4, 30)
x, y = next(batches)

In [22]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU


In [23]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        ## YOUR CODE HERE
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=self.n_hidden, 
                            num_layers=self.n_layers, dropout=self.drop_prob, 
                            batch_first=True)

        # Define a dropout layer
        ## YOUR CODE HERE
        self.dropout = nn.Dropout(self.drop_prob)

        # Define the final, fully-connected output layer
        ## YOUR CODE HERE
        self.linear = nn.Linear(in_features=self.n_hidden, out_features=len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        ## YOUR CODE HERE
        out, hidden = self.lstm(x.float(), hidden)

        # Pass through a dropout layer
        ## YOUR CODE HERE
        out = self.dropout(out)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        ## YOUR CODE HERE
        out = out.reshape(out.size()[0] * out.size()[1], self.n_hidden)


        # Put x through the fully-connected layer
        ## YOUR CODE HERE
        out = self.linear(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden    

In [24]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                now = datetime.now()
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                      "Time: ", dt_string)

In [25]:
n_hidden = 512 
n_layers = 2

chars = tuple(set(text))
net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(82, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=82, bias=True)
)


In [26]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/dict_chars.pickle', 'wb') as f:
    pickle.dump(chars, f)

In [27]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/dict_chars.pickle', 'rb') as f:
  c = pickle.load(f)

In [28]:
print(c)

('T', 'h', '"', '_', 'O', 'q', 'M', 'i', '9', 'o', '/', 'v', '*', 'L', 't', 'X', 'U', '[', 'x', 'Y', 'W', 'b', 'F', 'D', 'e', 'S', 'C', '>', 'N', '<', ':', 'r', ')', 'A', 'p', '4', 'k', 'y', 'f', '\n', '(', 'B', 'j', 'u', 's', '2', 'a', 'G', '0', 'z', 'n', 'w', 'K', ' ', ';', ']', 'I', 'l', '5', 'g', ',', '8', '1', 'J', 'Z', 'P', 'V', '?', 'c', 'R', 'E', 'Q', 'H', 'm', "'", '3', '6', '!', '-', '.', 'd', '7')


In [29]:
batch_size = 256
seq_length = 100 
n_epochs = 80

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=500)

Epoch: 14/80... Step: 500... Loss: 1.8649... Val Loss: 1.7833 Time:  17/01/2022 23:23:28
Epoch: 28/80... Step: 1000... Loss: 1.5954... Val Loss: 1.4797 Time:  17/01/2022 23:24:17
Epoch: 41/80... Step: 1500... Loss: 1.3695... Val Loss: 1.3599 Time:  17/01/2022 23:25:05
Epoch: 55/80... Step: 2000... Loss: 1.2714... Val Loss: 1.3037 Time:  17/01/2022 23:25:53
Epoch: 68/80... Step: 2500... Loss: 1.1658... Val Loss: 1.2853 Time:  17/01/2022 23:26:43


In [30]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/text_generator.pt" 
torch.save(net.state_dict(), path)


In [31]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/text_generator.pt" 


with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/dict_chars.pickle', 'rb') as f:
    chars_2 = pickle.load(f)

n_hidden = 512  
n_layers = 2
net_2 = CharRNN(chars_2, n_hidden, n_layers)
net_2.load_state_dict(torch.load(path))

<All keys matched successfully>

In [32]:
net_2.eval()

CharRNN(
  (lstm): LSTM(82, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=82, bias=True)
)

In [33]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [34]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [35]:
# Example of generated text
print(sample(net, 8000, prime='Ahora pase el personaje anterior y obtenga uno nuevo', top_k=20))

Ahora pase el personaje anterior y obtenga uno nuevo de
el a obscura y pila; pues todavia los mismos teucros--al en en seguida ligero gran voyo a la
necha infentilo en triste miembra y le acundaba del carro; y si
no lo riseran, dijo a los teucros, recopias a los amigos
de quien persucera a los teucros. Tambien
su hermano, para que le mata por las naves y a Antiloco, dirigiera
los mortales, ne se largante a un dios. Y si lo amada te sabe faso
venido, tensa en la mucerte cincuenta llona. ?Que interriris
los rodillos! Creto a Hector y a mi hijo seria
funesto de la batalla, ni es un dios, conterron, los
aqueos les fuitas por el pecho y el apatito, con el combate de
los dioses, te hizo infulgo que a la divina estaba ceyas lor donde se
ojos a ohillas y aquel, todo como el seno y anima con interignale
inselvacian con las hijas del guarria de grandes olas.>>

545 Asi dijo; y el heroe, se puso en la blondira del timo:.

191 Diomedes, entregandole con sus hijos y a la libado, y
con el corazon ac

In [36]:
text_ph = text.split('. ')

In [37]:
list_b1 = [ i.split(' ')[0].lower() for i in text_ph]
list_b2 = [ ' '.join(j.lower() for j in i.split(' ')[: 2]) for i in text_ph]
list_b3 = [ ' '.join(j.lower() for j in i.split(' ')[: 3]) for i in text_ph]
list_b4 = [ ' '.join(j.lower() for j in i.split(' ')[: 4]) for i in text_ph]

In [38]:
Counter(list_b1).most_common(10)

[('y', 331),
 ('el', 155),
 ('pero', 112),
 ('los', 81),
 ('como', 78),
 ('mas', 69),
 ('no', 66),
 ('en', 62),
 ('nombre', 59),
 ('a', 56)]

In [39]:
import re

In [40]:
list_of_begins = set()
for i in Counter(list_b1).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b2).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b3).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

In [41]:
list(list_of_begins)

['',
 'Mas',
 'En',
 'El',
 'Mas ?por que',
 'El guerrero cayo',
 'Eaobremos todos',
 'Y',
 'Y ahora',
 'Pero no',
 'Nombre',
 'Nombre patronimico de',
 'El hijo de',
 'Nombre patronimico',
 'No',
 ' ',
 'El atrida',
 'Y en',
 'Y si',
 'Nombre patronimico de\n',
 'Como',
 'Los',
 'Pero',
 'Y cuando',
 'Y el',
 'A',
 'Como el',
 'Fue muerto por']

In [42]:
# Removing the empty starting characters
list_of_begins.remove('')

In [43]:
# Removing the empty starting characters
list_of_begins.remove(' ')

In [44]:
list(list_of_begins)

['Mas',
 'En',
 'El',
 'Mas ?por que',
 'El guerrero cayo',
 'Eaobremos todos',
 'Y',
 'Y ahora',
 'Pero no',
 'Nombre',
 'Nombre patronimico de',
 'El hijo de',
 'Nombre patronimico',
 'No',
 'El atrida',
 'Y en',
 'Y si',
 'Nombre patronimico de\n',
 'Como',
 'Los',
 'Pero',
 'Y cuando',
 'Y el',
 'A',
 'Como el',
 'Fue muerto por']

In [45]:
import random

In [46]:

num_docs = 15000

path = '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/'

for i in range(num_docs):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')


17/01/2022 23:30:37 |  number of processed files: 0 0.0 %
17/01/2022 23:44:40 |  number of processed files: 200 1.3 %
17/01/2022 23:58:41 |  number of processed files: 400 2.7 %
18/01/2022 00:12:43 |  number of processed files: 600 4.0 %
18/01/2022 00:26:49 |  number of processed files: 800 5.3 %
18/01/2022 00:40:53 |  number of processed files: 1000 6.7 %
18/01/2022 00:54:58 |  number of processed files: 1200 8.0 %
18/01/2022 01:09:04 |  number of processed files: 1400 9.3 %
18/01/2022 01:23:12 |  number of processed files: 1600 10.7 %
18/01/2022 01:37:19 |  number of processed files: 1800 12.0 %
18/01/2022 01:51:30 |  number of processed files: 2000 13.3 %
18/01/2022 02:05:41 |  number of processed files: 2200 14.7 %
18/01/2022 02:19:51 |  number of processed files: 2400 16.0 %
18/01/2022 02:34:00 |  number of processed files: 2600 17.299999999999997 %
18/01/2022 02:48:11 |  number of processed files: 2800 18.7 %
18/01/2022 03:02:21 |  number of processed files: 3000 20.0 %
18/01/202

In [ ]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

# This is the code to generate (to determine how many have already been generated)

list_gen = glob.glob('/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/*.txt')
list_num = [ int(i.split('/')[-1].split('.txt')[0].split('doc_')[-1]) for i in list_gen]
max(list_num), min(list_num)


(5446, 0)

In [ ]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

num_docs = 15000

path = '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_es/'

for i in range(max(list_num), num_docs, 1):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')